<a href="https://colab.research.google.com/github/muzakkialfarisi/MACHINE-LEARNING/blob/master/Ekplor_Unigram_Bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import kebutuhan library python**








In [ ]:
import nltk
nltk.download('punkt') # untuk mengunduh package punkt, untuk tokenisasi.
from google.colab import drive
drive.mount('/content/drive/') # untuk dapat mengakses file / folder didalam drive
from collections import Counter # untuk sorting dictionary
import math # untuk dapat menggunakan operasi matematika

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive/


**Fungsi tokenisasi, sekaligus mengecilkan semua text**

In [ ]:
def nltk_lowercase(text): # menggunakan parameter dari gabungan variable artikel
  return nltk.sent_tokenize(text.lower()) # tokenisasi menjadi beberapa kalimat dan lowercase semua text

**Fungsi tokenisasi menjadi beberapa kalimat, lalu menjadi beberapa token.**

In [ ]:
def tokenisasi(artikel): # menggunakan variable return dari fungsi nltk_lowercase(-)
  gudang_kalimat = [] # array/list untuk menampung semua kalimat teks awal
  for kalimat in artikel: # loop per kalimat, tambahkan tag <s> di awal kalimat dan </s> di akhir kalimat
      mod_sentence = []
      sent_tokens = nltk.word_tokenize(kalimat)
      mod_sentence.append('<s>')
      for token in sent_tokens:
        mod_sentence.append(token) # kumpulan kata pada 1 kalimat
      mod_sentence.append('</s>')
      gudang_kalimat.append(mod_sentence) # kumpulan kalimat pada artikel
  return gudang_kalimat

**Fungsi perhitngan frekuensi kemunculan unigram**

In [ ]:
def freq_unigram(gudang_kalimat): # menggunakna variable return dari fungsi tokenisasi(-)
  freq_unigram = {} # dictionary untuk unigram
  for kalimat in gudang_kalimat:
    for kata in kalimat:
      if kata in freq_unigram:
          freq_unigram[kata] += 1 # kata sudah ada di dictionary, update frekuensinya
      else:
          freq_unigram[kata] = 1 # kata belum ada di dictionary 
  return freq_unigram

**Fungsi perhitngan probabilitas dari unigram**

In [ ]:
def prob_unigram(freq_unigram): # menggunakan variable return dari fungsi freq_unigram(-)
  total_freq = sum(freq_unigram[item] for item in freq_unigram) # mengitung total value dari frequensi unigram
  prob_unigram = {} # dictionary untuk probabilitas unigram
  for token in freq_unigram:
    prob_unigram[token] = freq_unigram[token]/total_freq # membagi nilai frequensi kata[token] dengan nilai total frequensi
  return prob_unigram

**Fungsi mencari 10 frequensi tertinggi unigram**

In [ ]:
def top10_freq_unigram(unigram): # menggunkana variabe return dari fungsi freq_unigram(-)
  top10_unigram = Counter(unigram) # Library python, sorting dictionary value secara descending
  return top10_unigram.most_common()

**Fungsi perhitungan frequensi kemunculan bigram**

In [ ]:
def freq_bigram(gudang_kalimat): # menggunakan variable return dari fungsi tokenisasi(-)
  freq_bigram = {} # dictionary untuk frequensi bigram
  for kalimat in gudang_kalimat:
    for i in range (1, len(kalimat)):
      curr_bigram = (kalimat[i-1], kalimat[i]) # memasangkan kata sebekumnya dengan kata yang sekarang
      if curr_bigram in freq_bigram:
        freq_bigram[curr_bigram] += 1 # bigram sudah ada di dictionary, update frekuensinya
      else:
        freq_bigram[curr_bigram] = 1 # bigram belum ada di dictionary
  return freq_bigram

**Fungsi perhitngan probabilitas dari bigram**

In [ ]:
def prob_bigram(gudang_kalimat): # menggunakan variable return dari fungsi tokenisasi(-)
  freq_uni = freq_unigram(gudang_kalimat) # memanggil fungsi freq_unigram(-) untuk mendapatkan nilai frequensi unigram
  freq_bi = freq_bigram(gudang_kalimat) # memanggil fungsi freq_bigram(-) untuk mendapatkan nilai frequensi bigram
  prob_bigram = {} # dictionary untuk probabilitas bigram
  for sentence in gudang_kalimat:
    for i in range (1, len(sentence)):
      curr_bigram = (sentence[i-1], sentence[i]) # memasangkan kata sebekumnya dengan kata yang sekarang
      if curr_bigram not in prob_bigram: # ketika pasangan bigram saat ini tdk dalam dictionary probabilitas bigram
        prob_bigram[curr_bigram] = freq_bi[curr_bigram]/freq_uni[sentence[i-1]] # membagi frequensi bigram saat ini dengan kata ke-1 dari pasangan bigram
  return prob_bigram

**Fungsi mencari 10 probabilitas tertinggi bigram**

In [ ]:
def top10_prob_bigram(bigram): # menggunkana variabe return dari fungsi prob_bigram(-)
  top = Counter(bigram) # Library python, sorting dictionary value secara descending
  return top.most_common()

**Fungsi Smoothing**

In [ ]:
def smoothing(uji, latih):
  step_1 = nltk_lowercase(uji) # tokenisasi kalimat uji dan mengecilkan semua huruf
  step_2 = tokenisasi(step_1) # memecah menjadi beberapa kalimat dan menambahkan tag <s> diawal dan tag </s> diakhir

  freq_1 = freq_unigram(latih) # menyimpan frequensi unigram dari data latih
  freq_2 = freq_bigram(latih) # menyimpan frequensi bigram dari data latih
  temp_prob = []
  for kalimat in step_2: # loop setiap kalimat di data uji
    probabilitas = 1
    temp = 1
    for i in range (1, len(kalimat)): # loop kata sepanjang kalimat didalam data uji
      curr_bigram = (kalimat[i-1], kalimat[i]) # penggabungan kata menjadi bigram dari data uji
      if curr_bigram in freq_2: # ketika bigram ada di dictionary
        pembilang = freq_2[curr_bigram] + 1
        penyebut = freq_1[kalimat[i-1]]+len(freq_1)
      else:
        if kalimat[i-1] in freq_1: # ketika bigram tdk ada di dic dan unigram ada di dic
          pembilang = 1
          penyebut = freq_1[kalimat[i-1]]+len(freq_1)
        else: # ketika bigram tdk ada di dic dan unigram tidak ada di dic
          pembilang = 1
          penyebut = 1+len(freq_1)
      probabilitas = pembilang/penyebut
      temp = temp*probabilitas # probabilitas per kalimat
    temp_prob.append(temp) # masukan array
  return temp_prob

**Fungsi Perplexity**

In [ ]:
def perplex(uji, prob):
  step_1 = nltk_lowercase(uji) # tokenisasi kalimat uji dan mengecilkan semua huruf
  step_2 = tokenisasi(step_1) # memecah menjadi beberapa kalimat dan menambahkan tag <s> diawal dan tag </s> diakhir
  temp_len, temp_per = [], []
  for kalimat in step_2:
    temp_len.append(len(kalimat)) # menyimpan panjang tiap kalimat dalam array
  for i in range(len(step_2)):
    temp_per.append(prob[i]**-(1/temp_len[i])) # formula perplexity = prob kalimat uji ^ - (1/panjang kalimatnya)
  return temp_per

**MAIN PROGRAM**

In [ ]:
# membaca file dari drive
a1 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article1.txt').read()
a2 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article2.txt').read()
a3 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article3.txt').read()
a4 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article4.txt').read()
a5 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article5.txt').read()
a6 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article6.txt').read()
a7 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article7.txt').read()
a8 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article8.txt').read()
a9 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article9.txt').read()
a10 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article10.txt').read()
a11 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article11.txt').read()
a12 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article12.txt').read()
a13 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article13.txt').read()
a14 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article14.txt').read()
a15 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article15.txt').read()
a16 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article16.txt').read()
a17 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article17.txt').read()
a18 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article18.txt').read()
a19 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article19.txt').read()
a20 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/article20.txt').read()

# menggabungkan artikel
article = a1 + a2 + a3 + a4 + a5 + a6 + a7 + a8 + a9 + a10 + a11 + a12 + a13 + a14 + a15 + a16 + a17 + a18 + a19 + a20

FileNotFoundError: ignored

In [ ]:
latih = tokenisasi(nltk_lowercase(article)) # fungsi tokenisasi, hasilnya diubah tokenisasi menjadi beberapa kalimat lalau menjadi token
file_uji = open('/content/drive/My Drive/Colab Notebooks/DataTest/uji.txt').read() # membaca file data uji

prob_uji = smoothing(file_uji, latih) # fungsi smoothing
prex_uji = perplex(file_uji,prob_uji) #fungsi perplexity

temp1 = 1
temp2 = 1

print('-- - TOPIK SERUPA - --')
for i in range(3):
  print('Probability Uji ke-',i+1,'=',prob_uji[i]) # print index ke 0 - 2
  temp1 = prob_uji[i]*temp1
print('Probabilitas Topik serupa :', temp1)
print('\n-- - TOPIK BEDA - --')
for i in range(3,6): 
  print('Probability Uji ke-',i-2,'=',prob_uji[i]) # print index ke 3 - 5
  temp2 = prob_uji[i]*temp2
print('Probabilitas Topik berbeda :', temp2)

In [ ]:
print('-- - TOPIK SERUPA - --')
for i in range(3):
  print('Perplexity Uji ke-',i+1,'=',prex_uji[i]) # print index ke 0 - 2

print('\n-- - TOPIK BEDA - --')
for i in range(3,6): 
  print('Perplexity Uji ke-',i-2,'=',prex_uji[i]) # print index ke 3 - 5

**UNIGRAM**

10 frequensi tertinggi unigram

In [ ]:
step1_unigram = freq_unigram(step2) # memanggil fungsi unutk menentukan probabilitas unigram dari data latih
top10_unigram = top10_freq_unigram(step1_unigram) # memanggil fungsi untuk sorting descending frequensi unigram
print('10 frequensi tertinggi unigram\n')
for i in range(10):
  print(top10_unigram[i])

**BIGRAM**

In [ ]:
step2_bigram = prob_bigram(step2) # memanggil fungsi unutk menentukan probabilitas bigram dari data latih
top10_bigram = top10_prob_bigram(step2_bigram) # memanggil fungsi untuk sorting descending probabilitas bigram
print('10 probabilitas tertinggi bigram\n')
for i in range(10):
  print(top10_bigram[i])
